<div style="background: linear-gradient(to right, #8E44AD, #2C3E50); padding: 20px; border-radius: 10px; text-align: center;">
    <h1 style="color: white; font-weight: bold; font-family: Arial, sans-serif;">
        Stability Analysis of Anomalies: Feature Selection and Correlation Filtering
    </h1>
</div>

In [1]:
from IPython.core.display import display, HTML

/var/folders/1x/xt1xj2hx4p74zncm_ggmnn880000gn/T/ipykernel_12476/2058709175.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


<div style="background: linear-gradient(to right, #34490F, #000000); padding: 15px;">
    <h4 style="color: white;"><strong>1. Data Description & Task</strong></h4>
    <p style="color: white;">In this section, we describe the dataset used for stability analysis, including the structure of trace data and the nature of anomalies. The task involves selecting relevant features that explain the detected anomalies.</p>
</div>

In [4]:
import pandas as pd
#We will work with the 1_1.csv file at first

trace1_1 = pd.read_csv('1_1.csv')
labels = pd.read_csv('labels.csv')

#we will use only the labels from trace_id = 1_1

labels_1_1 = labels[labels["trace_id"] == '1_1']
labels_1_1

,Unnamed: 0,trace_id,ano_id,ref_start,ref_end,ano_start,ano_end,ano_type
0,0,1_1,0,4247,6193,6193,7166,bursty_input
1,1,1_1,1,11425,13393,13393,14377,bursty_input
2,2,1_1,2,18659,20593,20593,21560,bursty_input
3,3,1_1,3,25849,27793,27793,28765,bursty_input
4,4,1_1,4,33049,34993,34993,35965,bursty_input
5,5,1_1,5,40257,42193,42193,43161,bursty_input


To identify an anomaly, we have to use (trace_id,ano_id)

In [7]:
trace1_1.head()

,Unnamed: 0,driver_BlockManager_disk_diskSpaceUsed_MB_value,driver_BlockManager_memory_memUsed_MB_value,driver_BlockManager_memory_onHeapMemUsed_MB_value,driver_BlockManager_memory_remainingMem_MB_value,driver_BlockManager_memory_remainingOnHeapMem_MB_value,driver_DAGScheduler_job_activeJobs_value,driver_DAGScheduler_messageProcessingTime_m15_rate,driver_DAGScheduler_messageProcessingTime_m1_rate,driver_DAGScheduler_messageProcessingTime_m5_rate,...,avg_executor_shuffleTotalBytesRead_count_1_diff,avg_executor_shuffleWriteTime_count_1_diff,avg_executor_threadpool_completeTasks_value_1_diff,avg_jvm_PS-MarkSweep_count_value_1_diff,avg_jvm_PS-MarkSweep_time_value_1_diff,avg_jvm_PS-Scavenge_count_value_1_diff,avg_jvm_PS-Scavenge_time_value_1_diff,avg_jvm_non-heap_committed_value_1_diff,avg_jvm_pools_Code-Cache_committed_value_1_diff,avg_jvm_pools_Metaspace_committed_value_1_diff
0,0,0.0,2100.0,2100.0,58346.0,58346.0,3.0,74.14452,34.41122,63.44280,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,2134.0,2134.0,58313.0,58313.0,3.0,74.14452,34.41122,63.44280,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,2200.0,2200.0,58247.0,58247.0,3.0,74.14452,34.41122,63.44280,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,2265.0,2265.0,58181.0,58181.0,4.0,74.14452,34.41122,63.44280,...,0.0,793296.0,5.5,0.0,0.0,0.5,16.5,0.0,0.0,0.0
4,4,0.0,2299.0,2299.0,58148.0,58148.0,4.0,73.88555,33.85063,62.84706,...,385719.5,363575.5,22.0,0.0,0.0,0.5,9.5,65536.0,65536.0,0.0


We set the index to the first column

In [10]:
trace1_1 = trace1_1.set_index(trace1_1.columns[0])
trace1_1.head()

,driver_BlockManager_disk_diskSpaceUsed_MB_value,driver_BlockManager_memory_memUsed_MB_value,driver_BlockManager_memory_onHeapMemUsed_MB_value,driver_BlockManager_memory_remainingMem_MB_value,driver_BlockManager_memory_remainingOnHeapMem_MB_value,driver_DAGScheduler_job_activeJobs_value,driver_DAGScheduler_messageProcessingTime_m15_rate,driver_DAGScheduler_messageProcessingTime_m1_rate,driver_DAGScheduler_messageProcessingTime_m5_rate,driver_DAGScheduler_messageProcessingTime_max,...,avg_executor_shuffleTotalBytesRead_count_1_diff,avg_executor_shuffleWriteTime_count_1_diff,avg_executor_threadpool_completeTasks_value_1_diff,avg_jvm_PS-MarkSweep_count_value_1_diff,avg_jvm_PS-MarkSweep_time_value_1_diff,avg_jvm_PS-Scavenge_count_value_1_diff,avg_jvm_PS-Scavenge_time_value_1_diff,avg_jvm_non-heap_committed_value_1_diff,avg_jvm_pools_Code-Cache_committed_value_1_diff,avg_jvm_pools_Metaspace_committed_value_1_diff
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,0.0,2100.0,2100.0,58346.0,58346.0,3.0,74.14452,34.41122,63.44280,17.925589,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,2134.0,2134.0,58313.0,58313.0,3.0,74.14452,34.41122,63.44280,17.925589,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,2200.0,2200.0,58247.0,58247.0,3.0,74.14452,34.41122,63.44280,17.925589,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,2265.0,2265.0,58181.0,58181.0,4.0,74.14452,34.41122,63.44280,17.925589,...,0.0,793296.0,5.5,0.0,0.0,0.5,16.5,0.0,0.0,0.0
4,0.0,2299.0,2299.0,58148.0,58148.0,4.0,73.88555,33.85063,62.84706,17.925589,...,385719.5,363575.5,22.0,0.0,0.0,0.5,9.5,65536.0,65536.0,0.0


In [12]:
#Let's look at the trace of anomaly 0 from trace1_1

#We retrieve the Time interval for the reference
ref_start_1_1_0 = labels_1_1["ref_start"][0]
ref_end_1_1_0 = labels_1_1["ref_end"][0]

#We retrieve the Time interval for the anomaly
ano_start_1_1_0 = labels_1_1["ano_start"][0]
ano_end_1_1_0 = labels_1_1["ano_end"][0]



#We retrieve the trace of the reference and the anomaly

trace1_1_0_ano = trace1_1.iloc[ano_start_1_1_0:ano_end_1_1_0,:]
trace1_1_0_ref = trace1_1.iloc[ref_start_1_1_0:ref_end_1_1_0,:]

print("the interval of the reference is from ", ref_start_1_1_0, " to ", ref_end_1_1_0, "excluded")
print("the interval of the anomaly is from ", ano_start_1_1_0, " to ", ano_end_1_1_0, "excluded")

trace1_1_0_ano

the interval of the reference is from  4247  to  6193 excluded
the interval of the anomaly is from  6193  to  7166 excluded


,driver_BlockManager_disk_diskSpaceUsed_MB_value,driver_BlockManager_memory_memUsed_MB_value,driver_BlockManager_memory_onHeapMemUsed_MB_value,driver_BlockManager_memory_remainingMem_MB_value,driver_BlockManager_memory_remainingOnHeapMem_MB_value,driver_DAGScheduler_job_activeJobs_value,driver_DAGScheduler_messageProcessingTime_m15_rate,driver_DAGScheduler_messageProcessingTime_m1_rate,driver_DAGScheduler_messageProcessingTime_m5_rate,driver_DAGScheduler_messageProcessingTime_max,...,avg_executor_shuffleTotalBytesRead_count_1_diff,avg_executor_shuffleWriteTime_count_1_diff,avg_executor_threadpool_completeTasks_value_1_diff,avg_jvm_PS-MarkSweep_count_value_1_diff,avg_jvm_PS-MarkSweep_time_value_1_diff,avg_jvm_PS-Scavenge_count_value_1_diff,avg_jvm_PS-Scavenge_time_value_1_diff,avg_jvm_non-heap_committed_value_1_diff,avg_jvm_pools_Code-Cache_committed_value_1_diff,avg_jvm_pools_Metaspace_committed_value_1_diff
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
6193,0.0,3595.0,3595.0,56852.0,56852.0,4.0,26.212091,25.608845,26.085407,9.057331,...,0.0,164744.5,1.0,0.0,0.0,1.0,65.0,0.0,0.0,0.0
6194,0.0,3727.0,3727.0,56720.0,56720.0,4.0,26.184322,25.256332,26.004660,9.057331,...,0.0,523780.5,3.0,0.0,0.0,1.5,79.5,0.0,0.0,0.0
6195,0.0,4095.0,4095.0,56352.0,56352.0,4.0,26.184322,25.256332,26.004660,9.057331,...,0.0,522475.5,2.5,0.0,0.0,1.5,72.0,0.0,0.0,0.0
6196,0.0,4411.0,4411.0,56036.0,56036.0,4.0,26.184322,25.256332,26.004660,9.057331,...,0.0,396993.0,2.0,0.0,0.0,1.0,46.0,0.0,0.0,0.0
6197,0.0,4727.0,4727.0,55720.0,55720.0,4.0,26.184322,25.256332,26.004660,9.057331,...,1255451.0,313234.5,20.5,0.0,0.0,0.5,20.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7161,0.0,2101.0,2101.0,58346.0,58346.0,3.0,26.277475,26.983374,26.451902,27.516640,...,65062.5,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7162,0.0,2134.0,2134.0,58313.0,58313.0,3.0,26.277475,26.983374,26.451902,27.516640,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7163,0.0,2199.0,2199.0,58247.0,58247.0,4.0,26.277475,26.983374,26.451902,27.516640,...,0.0,472806.5,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We've seen what the data looks like. let's start working with the whole dataset.

# Storing the whole relevant traces into a dictionnary

We've seen that each anomaly is identified by the tuple (trace_id,ano_id), this is why we are going to use a dictionnary for that

For each anomaly, we have two traces, a normal and an abnormal one. we will access each trace using a key like : `(trace_id, ano_id, 'normal')` or  `(trace_id, ano_id, 'abnormal')` 

In [18]:
traces = {}

for _ , row in labels.iterrows():
    trace_id = row["trace_id"]
    ano_id = row["ano_id"]

    ref_start = row["ref_start"]
    ref_end = row["ref_end"]

    ano_start = row["ano_start"]
    ano_end = row["ano_end"]

    # we retrieve the corresponding trace

    trace = pd.read_csv(f"{trace_id}.csv")
    
    # we obtain the normal and abnormal period, which are in the form of a dataframe
    normal_period = trace.iloc[ref_start:ref_end,:] 
    abnormal_period = trace.iloc[ano_start:ano_end,:]

    #we store in our dictionary the traces
    
    traces[(trace_id, ano_id,'normal')] = normal_period
    traces[(trace_id, ano_id,'abnormal')] = abnormal_period

In [19]:
#test
print("trace of anomaly 2 from trace 2_1")

#retrieve df of trace 2_1

labels_2_1 = labels[labels["trace_id"] == '2_1']

#We retrieve the Time interval for the anomaly

ano_start_2_1_2 = labels_2_1[labels_2_1["ano_id"] == 2]["ano_start"].values[0]
ano_end_2_1_2 = labels_2_1[labels_2_1["ano_id"] == 2]["ano_end"].values[0]

print("the interval of the anomaly is from ", ano_start_2_1_2, " to ", ano_end_2_1_2, "excluded")
print()
print("let's see if it matches with what we have in the dictionary") 

traces[('2_1', 2, 'abnormal')]

trace of anomaly 2 from trace 2_1
the interval of the anomaly is from  28708  to  29684 excluded

let's see if it matches with what we have in the dictionary


,Unnamed: 0,driver_BlockManager_disk_diskSpaceUsed_MB_value,driver_BlockManager_memory_memUsed_MB_value,driver_BlockManager_memory_onHeapMemUsed_MB_value,driver_BlockManager_memory_remainingMem_MB_value,driver_BlockManager_memory_remainingOnHeapMem_MB_value,driver_DAGScheduler_job_activeJobs_value,driver_DAGScheduler_messageProcessingTime_m15_rate,driver_DAGScheduler_messageProcessingTime_m1_rate,driver_DAGScheduler_messageProcessingTime_m5_rate,...,avg_executor_shuffleTotalBytesRead_count_1_diff,avg_executor_shuffleWriteTime_count_1_diff,avg_executor_threadpool_completeTasks_value_1_diff,avg_jvm_PS-MarkSweep_count_value_1_diff,avg_jvm_PS-MarkSweep_time_value_1_diff,avg_jvm_PS-Scavenge_count_value_1_diff,avg_jvm_PS-Scavenge_time_value_1_diff,avg_jvm_non-heap_committed_value_1_diff,avg_jvm_pools_Code-Cache_committed_value_1_diff,avg_jvm_pools_Metaspace_committed_value_1_diff
28708,28708,0.0,5038.0,5038.0,55409.0,55409.0,1.0,11.465651,11.917851,11.530488,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28709,28709,0.0,5134.0,5134.0,55312.0,55312.0,1.0,11.402129,10.964953,11.339906,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28710,28710,0.0,5175.0,5175.0,55272.0,55272.0,1.0,11.402129,10.964953,11.339906,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28711,28711,0.0,5175.0,5175.0,55272.0,55272.0,2.0,11.402129,10.964953,11.339906,...,0.0,0.0,0.0,0.0,0.0,1.0,22.5,0.0,0.0,0.0
28712,28712,0.0,5175.0,5175.0,55272.0,55272.0,2.0,11.402129,10.964953,11.339906,...,126336.0,622660.5,9.5,0.0,0.0,1.0,23.5,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29679,29679,0.0,5084.0,5084.0,55363.0,55363.0,1.0,9.436953,9.804560,8.772171,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29680,29680,0.0,5211.0,5211.0,55236.0,55236.0,1.0,9.436953,9.804560,8.772171,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29681,29681,0.0,5307.0,5307.0,55140.0,55140.0,2.0,9.436953,9.804560,8.772171,...,0.0,0.0,0.0,0.0,0.0,0.5,13.5,0.0,0.0,0.0
29682,29682,0.0,5402.0,5402.0,55045.0,55045.0,2.0,9.436953,9.804560,8.772171,...,0.0,812954.5,4.5,0.0,0.0,1.0,23.0,0.0,0.0,0.0


<div style="background: linear-gradient(to right, #C0392B, #000000); padding: 15px;">
    <h4 style="color: white;"><strong>4. Computing Feature Rewards Using Entropy-Based Metrics</strong></h4>
    <p style="color: white;">For each feature within each anomaly, we compute a reward score that quantifies the significance of the feature in explaining the anomaly. This is done by comparing the feature's behavior during normal and abnormal periods.</p>
</div>

In [21]:
import numpy as np


def class_entropy(normal_values, abnormal_values):
    """
    This function computes the class entropy of the normal and abnormal values
    """
    #We compute the number of normal and abnormal values
    n_normal = len(normal_values)
    n_abnormal = len(abnormal_values)
    
    #We compute the total number of values
    n_total = n_normal + n_abnormal
    
    #we compute the proportion of normal and abnormal values

    p_normal = n_normal/n_total
    p_abnormal = n_abnormal/n_total

    #We compute the class entropy
    class_entropy = -p_normal * np.log(p_normal) - p_abnormal * np.log(p_abnormal)

    return class_entropy

In [22]:
s1 = set([0,0,1,3,2])
s2 = set([1,0,0,3,6,7,8,9])
mixed = s1 & s2

print(s1 - mixed)

{2}


While implementing the function, we noticed a special case that need to be handled: When we only have 1 value for a feature throughout the trace. It's the only scenario where our segmentation entropy can be null despite adding the penalty term for mixed segment. we will assign a value of 0 for the reward, because such a feature don't give us any information about the desired anomaly.

In [24]:
# Let's implement the function of Segmentation entropy

import math
from collections import Counter

# Since we will have tuples with (count_for_normal, count_for_abnormal) for mixed values
# we need to implement a function to sum the values of the tuples and the values of the list

def sum_values_and_tuples(arr):
    total_sum = 0
    for item in arr:
        if isinstance(item, tuple):
            total_sum += sum(item)
        else:
            total_sum += item
    return total_sum


# We implement the initial function of Segmentation entropy

def segmentation_entropy(list_of_segments, total_count):
    """
    This function computes the segmentation entropy of a list of segments
    """

    # Reminder, a segment is a dictionary with keys "type", "values" and "counts"
    segmentation_entropy = 0
    for segment in list_of_segments:
        segment_size = sum_values_and_tuples(segment["counts"])
        p_i = segment_size / total_count
        segmentation_entropy -= p_i * math.log(p_i)
        if segment["type"] == "M":
            # If the segment is mixed, we need to compute the entropy of the mixed segment
            worst_case_entropy = worst_case_ordering_and_entropy(segment)
            segmentation_entropy += worst_case_entropy
    
    return segmentation_entropy

#We implement a function to give us the worst case entropy of a list of segments

def worst_case_ordering_and_entropy(segment):
    values = segment['values']
    counts = segment['counts']
    
    # Generate the worst-case sequence by alternating between Ns and As
    total_counts = [sum(count) for count in counts]
    total_count = sum(total_counts)
    result = []
    
    last_type = None
    while any(total_counts):
        for i in range(len(values)):
            if total_counts[i] > 0:
                if counts[i][0] > 0 and (last_type is None or last_type == 'A'):
                    result.append('N')
                    counts[i] = (counts[i][0] - 1, counts[i][1])
                    total_counts[i] -= 1
                    last_type = 'N'
                    break
                elif counts[i][1] > 0 and (last_type is None or last_type == 'N'):
                    result.append('A')
                    counts[i] = (counts[i][0], counts[i][1] - 1)
                    total_counts[i] -= 1
                    last_type = 'A'
                    break
        else:
            # If we can't find a different type, just add any available type
            for i in range(len(values)):
                if total_counts[i] > 0:
                    if counts[i][0] > 0:
                        result.append('N')
                        counts[i] = (counts[i][0] - 1, counts[i][1])
                        total_counts[i] -= 1
                        last_type = 'N'
                    elif counts[i][1] > 0:
                        result.append('A')
                        counts[i] = (counts[i][0], counts[i][1] - 1)
                        total_counts[i] -= 1
                        last_type = 'A'
                    break
    
    #print("the worst case sequence is: ", result)
    # Compute the segmentation entropy
    total_points = len(result)
    entropy = 0
    current_segment_length = 1
    
    for i in range(1, total_points):
        if result[i] != result[i - 1]:
            p = current_segment_length / total_points
            entropy -= p * math.log(p)
            current_segment_length = 1
        else:
            current_segment_length += 1
    
    # Add the last segment
    p = current_segment_length / total_points
    entropy -= p * math.log(p)
    
    #print("the worst case entropy is: ", entropy)
    
    return entropy


    
def compute_sequential_entropy(normal_seq, abnormal_seq):
    """
    Compute sequential entropy for merged normal and abnormal sequences.
    
    Args:
        normal_seq (list): List of normal class values.
        abnormal_seq (list): List of abnormal class values.
        
    Returns:
        dict: {
            "segments": List of segments (type, values),
            "segmentation_entropy": Segmentation entropy,
            "worst_case_entropy": Worst-case entropy for mixed segments,
        }
    """

    # Count occurrences of each value (will be useful for worst-case entropy)
    normal_counter = Counter(normal_seq)
    abnormal_counter = Counter(abnormal_seq)

    total_count = sum_values_and_tuples(normal_counter.values()) + sum_values_and_tuples(abnormal_counter.values())

    # Convert counters to sets of keys
    normal_set = set(normal_counter.keys())
    abnormal_set = set(abnormal_counter.keys())

    # Perform set operations
    mixed_values = normal_set & abnormal_set  # Intersection: values in both N and A
    normal_only = normal_set - mixed_values  # Values only in N
    abnormal_only = abnormal_set - mixed_values  # Values only in A

    # Create a merged list with labels and counts
    merged = [(val, 'N', normal_counter[val]) for val in normal_only] + \
            [(val, 'A', abnormal_counter[val]) for val in abnormal_only] + \
            [(val, 'M', (normal_counter[val], abnormal_counter[val])) for val in mixed_values]

    # Sort by value
    merged.sort(key=lambda x: x[0])


    #print("the sorted merged list of values and labels is: ", merged)

    # Step 2: Create segments (normal, abnormal, and mixed)
    segments = []
    current_segment = {"type": None, "values": [], "counts": []}
    
    for value, label, count in merged:
        if current_segment["type"] is None:
            current_segment["type"] = label
            current_segment["values"].append(value)
            current_segment["counts"].append(count)
        elif current_segment["type"] == label:
            current_segment["values"].append(value)
            current_segment["counts"].append(count)
        else:
            # Append the current segment
            segments.append(current_segment)
            # Start a new segment
            current_segment = {"type": label, "values": [value], "counts": [count]}

    # Append the last segment
    if current_segment["values"]:
        segments.append(current_segment)

    # Segments is a list of dictionaries with keys "type" and "values"

    #print("the segments are: ", segments)

    # Step 3: Compute segmentation entropy
    #total_points = len(merged)

    seg_entropy = segmentation_entropy(segments, total_count)


    #with the list of mixed segments, we can compute the worst case entropy

    return seg_entropy



In [25]:
#test

abnormal_example = [0,0,1,4,2,9,8,7,7,3,2]
normal_example = [1,0,0,3,6,7,3,0,0,0,8,9]


compute_sequential_entropy(normal_example, abnormal_example)

6.15740710637156

Now that we have the two entropies, let's compute the reward

In [27]:
#let's compute the reward

def compute_reward(normal_seq, abnormal_seq):
    """
    Compute the reward for a given pair of normal and abnormal sequences.
    
    Args:
        normal_seq (list): List of normal class values.
        abnormal_seq (list): List of abnormal class values.
        
    Returns:
        float: Reward value.
    """
    # Compute sequential entropy
    seq_entropy = compute_sequential_entropy(normal_seq, abnormal_seq)
    #print("the sequential entropy is: ", seq_entropy)
    
    # Compute class entropy
    cl_entropy = class_entropy(normal_seq, abnormal_seq)
    #print("the class entropy is: ", cl_entropy)
    
    # Compute reward
    if seq_entropy == 0:
        return 0

    reward = cl_entropy/seq_entropy
    
    return reward


In [28]:
#test
abnormal_example = [0,0,1,4,2,9,8,7,7,3,2]
normal_example = [1,0,0,3,6,7,3,0,0,0,8,9]

compute_reward(normal_example, abnormal_example)

0.11241772567644592

In [29]:
#Let's compute the reward for each feature for the first anomaly of trace 1_1

#We retrieve the normal and abnormal values

normal_values = traces[('1_1', 0, 'normal')].values
abnormal_values = traces[('1_1', 0, 'abnormal')].values

#We compute the reward for each feature

rewards = {}

for i in range(normal_values.shape[1]):
    normal_feature = normal_values[:,i]
    abnormal_feature = abnormal_values[:,i]
    rewards[i] = compute_reward(normal_feature, abnormal_feature)

#show the sequence when reward is inf

rewards

{0: 1.0,
 1: 0.11195325807707575,
 2: 0.02570585433587741,
 3: 0.02495930779170318,
 4: 0.023378420830705036,
 5: 0.023917180732417255,
 6: 0.11195325807707575,
 7: 0.2375995116153921,
 8: 0.12810429554768496,
 9: 0.15040402854658222,
 10: 0.10508223002363111,
 11: 0.039240805604943994,
 12: 0.37579859105729396,
 13: 0.09391508282987998,
 14: 0.012830509462981097,
 15: 0.030389559059704998,
 16: 0.048903806362763855,
 17: 0.0664091439951501,
 18: 0.08338883038263932,
 19: 0.11625774000700333,
 20: 0.07153862990593697,
 21: 0.11195325807707575,
 22: 0.11195325807707575,
 23: 0.15811027659231283,
 24: 0.12634117344299856,
 25: 0.09029261783581954,
 26: 0.09191334387833898,
 27: 0.028022936219529886,
 28: 0.011403311836291818,
 29: 0.10998723260420061,
 30: 0.038726452992562825,
 31: 0.019888360992845182,
 32: 0.04870870518450333,
 33: 0.005539754285600231,
 34: 0.010440203094070096,
 35: 0.052036790201264636,
 36: 0.002326179211864977,
 37: 0.1875186999434784,
 38: 0.09982693909867317,
 

In [96]:
def compute_rewards(trace, labels, trace_id):
    """
    Compute the reward for each feature of each anomaly in a given trace.
    """
    rewards = {}

    # ✅ Step 1: Ensure first column is removed if it is unnamed
    if trace.columns[0].startswith("Unnamed"):
        trace = trace.iloc[:, 1:]  # Remove first column
    if labels.columns[0].startswith("Unnamed"):
        labels = labels.iloc[:, 1:]  # Remove first column

    # ✅ Step 2: Filter labels based on the provided trace_id
    filtered_labels = labels[labels["trace_id"] == trace_id]  

    print(f"\n📌 Computing rewards for trace: {trace_id}")
    print(f"📝 Filtered labels:\n{filtered_labels}")

    # ✅ Step 3: Iterate over anomalies in the current trace
    for _, row in filtered_labels.iterrows():
        ano_id = row["ano_id"]

        # ✅ Convert indices to integers
        ref_start, ref_end = int(row["ref_start"]), int(row["ref_end"])
        ano_start, ano_end = int(row["ano_start"]), int(row["ano_end"])

        max_index = len(trace) - 1  # Last valid index

        # ✅ Ensure indices are within valid range
        if ref_start > max_index or ref_end > max_index or ano_start > max_index or ano_end > max_index:
            print(f"⚠️ Error: Indices out of bounds for trace {trace_id}, ano_id {ano_id}. Skipping this anomaly.")
            continue  # Skip anomaly

        if ref_start < 0 or ref_end < 0 or ano_start < 0 or ano_end < 0:
            print(f"⚠️ Error: Negative indices detected for trace {trace_id}, ano_id {ano_id}. Skipping this anomaly.")
            continue  # Skip anomaly

        # ✅ Step 4: Extract normal and anomalous periods
        normal_period = trace.iloc[ref_start:ref_end, :]
        abnormal_period = trace.iloc[ano_start:ano_end, :]

        if abnormal_period.empty:
            print(f"⚠️ Warning: Anomalous period for (trace_id, ano_id) = ({trace_id}, {ano_id}) is empty! Skipping...")
            continue  # Skip anomaly

        print(f"\n🔍 Vérification des valeurs moyennes pour (trace_id, ano_id) = ({trace_id}, {ano_id})")
        print(f"   - Moyenne période normale: {normal_period.mean()}")
        print(f"   - Moyenne période anormale: {abnormal_period.mean()}")
        print(f"   - Différences absolues: {np.abs(normal_period.mean() - abnormal_period.mean())}")

        # Convert to NumPy arrays
        normal_values = normal_period.values
        abnormal_values = abnormal_period.values

        # ✅ Step 5: Compute rewards for each feature
        for i in range(normal_values.shape[1]):  # Loop through features
            normal_feature = normal_values[:, i]
            abnormal_feature = abnormal_values[:, i]
            rewards[(trace_id, ano_id, i)] = compute_reward(normal_feature, abnormal_feature)

    return rewards

In [98]:
#Un peu long

# let's test the function

rewards_1_1 = compute_rewards(trace1_1, labels_1_1, "1_1")


📌 Computing rewards for trace: 1_1
📝 Filtered labels:
  trace_id  ano_id  ref_start  ref_end  ano_start  ano_end      ano_type
0      1_1       0       4247     6193       6193     7166  bursty_input
1      1_1       1      11425    13393      13393    14377  bursty_input
2      1_1       2      18659    20593      20593    21560  bursty_input
3      1_1       3      25849    27793      27793    28765  bursty_input
4      1_1       4      33049    34993      34993    35965  bursty_input
5      1_1       5      40257    42193      42193    43161  bursty_input

🔍 Vérification des valeurs moyennes pour (trace_id, ano_id) = (1_1, 0)
   - Moyenne période normale: driver_BlockManager_disk_diskSpaceUsed_MB_value               0.000000
driver_BlockManager_memory_memUsed_MB_value                2201.670092
driver_BlockManager_memory_onHeapMemUsed_MB_value          2201.693217
driver_BlockManager_memory_remainingMem_MB_value          58245.205036
driver_BlockManager_memory_remainingOnHeapMem_MB

In [99]:
rewards_2_2 = compute_rewards(trace1_2, labels, "1_2")


📌 Computing rewards for trace: 1_2
📝 Filtered labels:
   trace_id  ano_id  ref_start  ref_end  ano_start  ano_end      ano_type
6       1_2       0       5055     6989       6989     7956  bursty_input
7       1_2       1      13025    14989      14989    15971  bursty_input
8       1_2       2      21043    22989      22989    23962  bursty_input
9       1_2       3      29033    30989      30989    31967  bursty_input
10      1_2       4      37045    38989      38989    39961  bursty_input

🔍 Vérification des valeurs moyennes pour (trace_id, ano_id) = (1_2, 0)
   - Moyenne période normale: driver_BlockManager_disk_diskSpaceUsed_MB_value               0.000000
driver_BlockManager_memory_memUsed_MB_value                2188.822647
driver_BlockManager_memory_onHeapMemUsed_MB_value          2189.425026
driver_BlockManager_memory_remainingMem_MB_value          58257.154602
driver_BlockManager_memory_remainingOnHeapMem_MB_value    58256.644778
                                            

In [101]:
rewards_2_2

{('1_2', 0, 0): 0.11203451682928842,
 ('1_2', 0, 1): 0.03371243530065346,
 ('1_2', 0, 2): 0.033957462519503226,
 ('1_2', 0, 3): 0.033941051857173526,
 ('1_2', 0, 4): 0.03611605856245084,
 ('1_2', 0, 5): 0.11203451682928842,
 ('1_2', 0, 6): 0.32654676816871786,
 ('1_2', 0, 7): 0.17399839028258215,
 ('1_2', 0, 8): 0.08145680699844882,
 ('1_2', 0, 9): 0.10150258758811696,
 ('1_2', 0, 10): 0.029984735392111102,
 ('1_2', 0, 11): 0.10270713954826098,
 ('1_2', 0, 12): 0.05155772259757897,
 ('1_2', 0, 13): 0.02199343144621308,
 ('1_2', 0, 14): 0.011894241696293705,
 ('1_2', 0, 15): 0.0354569990631991,
 ('1_2', 0, 16): 0.09826381044848813,
 ('1_2', 0, 17): 0.07185436035626626,
 ('1_2', 0, 18): 0.1645965463169528,
 ('1_2', 0, 19): 0.06577003070382031,
 ('1_2', 0, 20): 0.11203451682928842,
 ('1_2', 0, 21): 0.11203451682928842,
 ('1_2', 0, 22): 0.0692565104091883,
 ('1_2', 0, 23): 0.16006596926940142,
 ('1_2', 0, 24): 0.1513267795951388,
 ('1_2', 0, 25): 0.12868469549434164,
 ('1_2', 0, 26): 0.019

<!-- Correlation Analysis and Final Feature Selection -->
<div style="background: linear-gradient(to right, #27AE60, #2C3E50); padding: 15px;">
    <h4 style="color: white;"><strong>Correlation Analysis and Final Feature Selection</strong></h4>
    <p style="color: white;">
        In these combined steps, we perform a comprehensive correlation analysis and finalize feature selection:
    </p>
    <ul style="color: white; font-size: 1em;">
        <li><strong>Computing Feature Correlation Matrices:</strong> Calculate the correlation matrices for the selected features to identify relationships between them.</li>
        <li><strong>Building Correlation Graphs for Feature Clustering:</strong> Construct graphs where features are nodes, and edges represent strong correlations based on a defined threshold.</li>
        <li><strong>Detecting Correlated Feature Clusters:</strong> Identify clusters of highly correlated features within each correlation graph, indicating groups of related features.</li>
        <li><strong>Selecting the Best Feature per Cluster:</strong> From each identified cluster, select the feature with the highest reward score to ensure the most significant and non-redundant features are retained.</li>
        <li><strong>Organizing and Displaying Final Features for Anomalies:</strong> Compile and present the final set of selected features for each anomaly, providing a clear and concise summary of the most relevant features.</li>
    </ul>
</div>


In [53]:
pip install kneed

Note: you may need to restart the kernel to use updated packages.


In [58]:
from collections import defaultdict
import numpy as np
from kneed import KneeLocator

def filter_and_sort_rewards(rewards):
    """
    Creates a sorted dictionary of rewards by (trace_id, ano_id).

    Args:
        rewards (dict): Dictionary with (trace_id, ano_id, feature) as keys and rewards as values.

    Returns:
        dict: Dictionary with (trace_id, ano_id) as keys and a sorted list of features and their rewards.
    """
    filtered_rewards = defaultdict(list)
    for (trace_id, ano_id, feature), reward in rewards.items():
        filtered_rewards[(trace_id, ano_id)].append((feature, reward))
    for key in filtered_rewards:
        filtered_rewards[key] = sorted(filtered_rewards[key], key=lambda x: x[1], reverse=True)
    return filtered_rewards

def find_all_elbows_threshold(reward_values, min_relative_drop=0.05):
    """
    Trouve **tous** les points de coude (elbows) et sélectionne le plus pertinent.

    Args:
        reward_values (list): Liste des valeurs de récompense triées **dans l'ordre décroissant**.
        min_relative_drop (float): Seuil minimal de variation relative pour considérer un coude.

    Returns:
        float: Le seuil optimal sélectionné.
    """
    if len(reward_values) < 2:
        return 0  # Cas trivial

    # Trouver **tous** les coudes possibles
    kneedle = KneeLocator(range(len(reward_values)), reward_values, curve="convex", direction="decreasing", online=True)
    all_knees = list(sorted(kneedle.all_knees))  # Convertir en liste triée

    if not all_knees:
        return 0  # Aucun coude détecté

    # Calculer les variations relatives entre les points détectés
    elbow_candidates = []
    for idx in all_knees:
        if idx == 0 or idx >= len(reward_values) - 1:
            continue  # Ignorer le premier et dernier point (cas bord)

        drop = abs(reward_values[idx] - reward_values[idx + 1]) / reward_values[idx]
        if drop >= min_relative_drop:  # Seuil de variation minimale pour considérer un coude
            elbow_candidates.append((idx, reward_values[idx], drop))

    if not elbow_candidates:
        return reward_values[all_knees[-1]]  # Prendre le dernier coude détecté si aucun ne respecte le seuil

    # Sélectionner le coude avec la **plus forte chute relative**
    best_elbow = max(elbow_candidates, key=lambda x: x[2])
    return best_elbow[1]  # Retourne la valeur seuil optimale

def apply_reward_filtering(filtered_rewards, max_features=20):
    """
    Applies Reward Leap Filtering using the Elbow Method and an optional fixed threshold.

    Args:
        filtered_rewards (dict): Dictionary of sorted rewards by (trace_id, ano_id).
        max_features (int): Maximum number of features allowed. A fixed threshold is applied if exceeded.

    Returns:
        dict: Final filtered rewards.
        dict: Number of features kept per anomaly.
        dict: Thresholds used per anomaly.
    """
    final_filtered_rewards = {}
    features_per_anomaly = defaultdict(int)
    thresholds_per_anomaly = defaultdict(list)

    for (trace_id, ano_id), features in filtered_rewards.items():
        selected_features = []
        prev_reward = None
        
        # Extract reward values for threshold calculation
        reward_values = [reward for _, reward in features]
        threshold = find_all_elbows_threshold(reward_values)
        thresholds_per_anomaly[ano_id].append(threshold)

        # Apply reward filtering
        for feature, reward in features:
            if prev_reward is not None and abs(prev_reward - reward) > threshold:
                break  # Stop when the threshold is exceeded
            selected_features.append((feature, reward))
            prev_reward = reward

        # If more than max_features are kept, apply a fixed threshold
        if len(selected_features) > max_features:
            print(f"⚠️ More than {max_features} features retained for (trace_id, ano_id)=({trace_id}, {ano_id})! Applying fixed threshold = 0.1")
            selected_features = []
            prev_reward = None
            threshold = 0.1  # Fixed threshold
            thresholds_per_anomaly[ano_id] = threshold  # Update the final threshold used

            for feature, reward in features:
                if prev_reward is not None and abs(prev_reward - reward) > threshold:
                    break  # Stop when the fixed threshold is exceeded
                selected_features.append((feature, reward))
                prev_reward = reward

        final_filtered_rewards[(trace_id, ano_id)] = selected_features
        features_per_anomaly[ano_id] += len(selected_features)

        print(f"For (trace_id, ano_id)=({trace_id}, {ano_id}), {len(selected_features)} features retained with a threshold of {threshold:.4f}.")
    
    return final_filtered_rewards, features_per_anomaly, thresholds_per_anomaly

def summarize_feature_filtering(features_per_anomaly, thresholds_per_anomaly):
    """
    Summarizes the number of features retained and the thresholds used for each anomaly.

    Args:
        features_per_anomaly (dict): Dictionary with the number of features retained per anomaly.
        thresholds_per_anomaly (dict): Dictionary with the thresholds used per anomaly.

    Returns:
        None
    """
    print("\n📊 Summary of retained features per anomaly and thresholds used:")
    for ano_id, count in features_per_anomaly.items():
        avg_threshold = np.mean(thresholds_per_anomaly[ano_id]) if thresholds_per_anomaly[ano_id] else 0
        print(f"  Anomaly {ano_id}: {count} features retained | Final threshold: {avg_threshold:.4f}")

def display_final_features(final_filtered_rewards):
    """
    Displays the final retained features for each (trace_id, ano_id).

    Args:
        final_filtered_rewards (dict): Dictionary of final retained features and their rewards.

    Returns:
        None
    """
    print("\n📋 Final retained features:")
    for (trace_id, ano_id), features in final_filtered_rewards.items():
        print(f"\n(trace_id, ano_id)=({trace_id}, {ano_id}):")
        for feature, reward in features:
            print(f"  Feature: {feature}, Reward: {reward}")

# Example usage with your rewards dictionary
filtered_rewards = filter_and_sort_rewards(rewards_1_1)
final_filtered_rewards, features_per_anomaly, thresholds_per_anomaly = apply_reward_filtering(filtered_rewards)
summarize_feature_filtering(features_per_anomaly, thresholds_per_anomaly)
display_final_features(final_filtered_rewards)

For (trace_id, ano_id)=(1_1, 0), 5 features retained with a threshold of 0.0419.
For (trace_id, ano_id)=(1_1, 1), 2 features retained with a threshold of 0.0376.
For (trace_id, ano_id)=(1_1, 2), 2 features retained with a threshold of 0.0560.
For (trace_id, ano_id)=(1_1, 3), 4 features retained with a threshold of 0.0596.
For (trace_id, ano_id)=(1_1, 4), 1 features retained with a threshold of 0.0418.
For (trace_id, ano_id)=(1_1, 5), 1 features retained with a threshold of 0.0511.

📊 Summary of retained features per anomaly and thresholds used:
  Anomaly 0: 5 features retained | Final threshold: 0.0419
  Anomaly 1: 2 features retained | Final threshold: 0.0376
  Anomaly 2: 2 features retained | Final threshold: 0.0560
  Anomaly 3: 4 features retained | Final threshold: 0.0596
  Anomaly 4: 1 features retained | Final threshold: 0.0418
  Anomaly 5: 1 features retained | Final threshold: 0.0511

📋 Final retained features:

(trace_id, ano_id)=(1_1, 0):
  Feature: 193, Reward: 1.0
  Feature

In [59]:
import pandas as pd
import numpy as np
import networkx as nx
from collections import defaultdict

def extract_features_per_anomaly(final_filtered_rewards):
    """
    Extracts the features retained for each anomaly and organizes them by (trace_id, ano_id).
    
    Args:
        final_filtered_rewards (dict): Dictionary of features retained with rewards for each anomaly.
        
    Returns:
        dict: Features retained per anomaly in a structured format.
    """
    features_per_anomaly = {}
    for (trace_id, ano_id), features in final_filtered_rewards.items():
        feature_names = [feat for feat, _ in features]
        features_per_anomaly[(trace_id, ano_id)] = feature_names
    print("📊 Selected features before clustering:", features_per_anomaly)
    return features_per_anomaly

def compute_correlation_matrices(features_per_anomaly, correlation_matrices):
    """
    Computes the correlation matrix for each anomaly based on selected features.
    
    Args:
        features_per_anomaly (dict): Selected features per anomaly.
        correlation_matrices (dict): Dictionary to store correlation matrices.
        
    Returns:
        dict: Updated dictionary containing correlation matrices for each anomaly.
    """
    for (trace_id, ano_id), feature_indices in features_per_anomaly.items():
        trace_df = pd.read_csv(f"{trace_id}.csv")  # Load data
        
        # Validate feature indices or names
        if all(isinstance(idx, int) for idx in feature_indices):
            feature_names = [trace_df.columns[idx] for idx in feature_indices if idx < len(trace_df.columns)]
        else:
            feature_names = feature_indices  # If already names, no conversion needed
        
        # Check for missing features
        missing_features = [f for f in feature_names if f not in trace_df.columns]
        if missing_features:
            print(f"⚠️ Missing features in {trace_id}.csv: {missing_features}")
            continue  # Skip to the next anomaly
        
        # Filter and standardize data
        filtered_df = trace_df[feature_names]
        filtered_df = (filtered_df - filtered_df.mean()) / filtered_df.std()

        # Compute correlation matrix
        corr_matrix = filtered_df.corr()
        correlation_matrices[(trace_id, ano_id)] = corr_matrix
        print(f"📌 Correlation matrix computed for (trace_id, ano_id) = ({trace_id}, {ano_id})")
        print(corr_matrix)
    return correlation_matrices

def build_correlation_graphs(correlation_matrices, correlation_threshold=0.85):
    """
    Constructs correlation graphs for features exceeding the specified correlation threshold.
    
    Args:
        correlation_matrices (dict): Correlation matrices for each anomaly.
        correlation_threshold (float): Threshold to consider two features as correlated.
        
    Returns:
        dict: Graphs of correlated features for each anomaly.
    """
    correlation_graphs = {}
    for (trace_id, ano_id), corr_matrix in correlation_matrices.items():
        G = nx.Graph()  # Initialize a graph
        features = corr_matrix.columns
        G.add_nodes_from(features)  # Add features as nodes

        # Add edges for correlated features
        for i in range(len(features)):
            for j in range(i + 1, len(features)):
                if abs(corr_matrix.iloc[i, j]) > correlation_threshold:
                    G.add_edge(features[i], features[j])

        correlation_graphs[(trace_id, ano_id)] = G
        print(f"📊 Graph constructed for (trace_id, ano_id) = ({trace_id}, {ano_id}) with {len(G.nodes)} nodes.")
    return correlation_graphs

def detect_feature_clusters(correlation_graphs):
    """
    Detects clusters of correlated features for each anomaly.
    
    Args:
        correlation_graphs (dict): Graphs of correlated features for each anomaly.
        
    Returns:
        dict: Clusters of correlated features for each anomaly.
    """
    feature_clusters = {}
    for (trace_id, ano_id), G in correlation_graphs.items():
        clusters = list(nx.connected_components(G))  # Identify connected components (clusters)
        feature_clusters[(trace_id, ano_id)] = clusters
        print(f"📌 {len(clusters)} clusters detected for (trace_id, ano_id) = ({trace_id}, {ano_id})")
    return feature_clusters

def select_best_features(feature_clusters, selected_features):
    """
    Selects the best feature from each cluster based on the highest reward.
    
    Args:
        feature_clusters (dict): Clusters of correlated features.
        selected_features (dict): Features and their rewards for each anomaly.
        
    Returns:
        dict: Final retained features after removing redundancies.
    """
    final_features = {}
    for (trace_id, ano_id), clusters in feature_clusters.items():
        retained_features = []
        feature_rewards = dict(selected_features.get((trace_id, ano_id), []))
        
        for cluster in clusters:
            # Choose the feature with the highest reward in the cluster
            best_feature = max(cluster, key=lambda f: feature_rewards.get(f, 0))
            retained_features.append(best_feature)  # Stocker uniquement le nom de la feature
        
        final_features[(trace_id, ano_id)] = retained_features
        print(f"✅ {len(retained_features)} features retained after clustering for (trace_id, ano_id) = ({trace_id}, {ano_id})")
    
    return final_features


def display_final_summary(final_features):
    """
    Displays the final retained features for each anomaly after clustering.
    
    Args:
        final_features (dict): Final retained features after redundancy removal.
        
    Returns:
        None
    """
    print("\n📊 Final summary of retained features after clustering:")
    for (trace_id, ano_id), features in final_features.items():
        print(f"\n(trace_id, ano_id) = ({trace_id}, {ano_id}) :")
        for feature in features:  # Correction ici : on boucle simplement sur les features
            print(f"  - Feature: {feature}")

def organize_anomalies_features(final_features):
    """
    Organizes the final features for each anomaly into a dictionary.
    
    Args:
        final_features (dict): Final retained features after redundancy removal.
        
    Returns:
        dict: Dictionary mapping each anomaly to its final features.
    """
    anomalies_features = {}
    for (trace_id, ano_id), features in final_features.items():
        anomalies_features[(trace_id, ano_id)] = [feat for feat in features]  # On s'assure qu'on ne garde que les noms
    print(anomalies_features)
    return anomalies_features

In [60]:
features_per_anomaly = extract_features_per_anomaly(final_filtered_rewards)

📊 Selected features before clustering: {('1_1', 0): [193, 194, 186, 174, 199], ('1_1', 1): [175, 200], ('1_1', 2): [175, 200], ('1_1', 3): [175, 200, 174, 199], ('1_1', 4): [187], ('1_1', 5): [187]}


In [61]:
correlation_matrices = compute_correlation_matrices(features_per_anomaly, {})

📌 Correlation matrix computed for (trace_id, ano_id) = (1_1, 0)
                                            avg_jvm_pools_PS-Old-Gen_max_value  \
avg_jvm_pools_PS-Old-Gen_max_value                                         NaN   
avg_jvm_pools_PS-Old-Gen_usage_value                                       NaN   
avg_jvm_pools_Metaspace_usage_value                                        NaN   
avg_jvm_direct_used_value                                                  NaN   
avg_jvm_pools_PS-Survivor-Space_used_value                                 NaN   

                                            avg_jvm_pools_PS-Old-Gen_usage_value  \
avg_jvm_pools_PS-Old-Gen_max_value                                           NaN   
avg_jvm_pools_PS-Old-Gen_usage_value                                    1.000000   
avg_jvm_pools_Metaspace_usage_value                                    -0.445719   
avg_jvm_direct_used_value                                              -0.445378   
avg_jvm_pools_PS-Surviv

In [62]:
correlation_graphs = build_correlation_graphs(correlation_matrices)

📊 Graph constructed for (trace_id, ano_id) = (1_1, 0) with 5 nodes.
📊 Graph constructed for (trace_id, ano_id) = (1_1, 1) with 2 nodes.
📊 Graph constructed for (trace_id, ano_id) = (1_1, 2) with 2 nodes.
📊 Graph constructed for (trace_id, ano_id) = (1_1, 3) with 4 nodes.
📊 Graph constructed for (trace_id, ano_id) = (1_1, 4) with 1 nodes.
📊 Graph constructed for (trace_id, ano_id) = (1_1, 5) with 1 nodes.


In [63]:
feature_clusters = detect_feature_clusters(correlation_graphs)

📌 5 clusters detected for (trace_id, ano_id) = (1_1, 0)
📌 1 clusters detected for (trace_id, ano_id) = (1_1, 1)
📌 1 clusters detected for (trace_id, ano_id) = (1_1, 2)
📌 3 clusters detected for (trace_id, ano_id) = (1_1, 3)
📌 1 clusters detected for (trace_id, ano_id) = (1_1, 4)
📌 1 clusters detected for (trace_id, ano_id) = (1_1, 5)


In [64]:
final_features = select_best_features(feature_clusters, final_filtered_rewards)

✅ 5 features retained after clustering for (trace_id, ano_id) = (1_1, 0)
✅ 1 features retained after clustering for (trace_id, ano_id) = (1_1, 1)
✅ 1 features retained after clustering for (trace_id, ano_id) = (1_1, 2)
✅ 3 features retained after clustering for (trace_id, ano_id) = (1_1, 3)
✅ 1 features retained after clustering for (trace_id, ano_id) = (1_1, 4)
✅ 1 features retained after clustering for (trace_id, ano_id) = (1_1, 5)


In [67]:
display_final_summary(final_features)


📊 Final summary of retained features after clustering:

(trace_id, ano_id) = (1_1, 0) :
  - Feature: avg_jvm_pools_PS-Old-Gen_max_value
  - Feature: avg_jvm_pools_PS-Old-Gen_usage_value
  - Feature: avg_jvm_pools_Metaspace_usage_value
  - Feature: avg_jvm_direct_used_value
  - Feature: avg_jvm_pools_PS-Survivor-Space_used_value

(trace_id, ano_id) = (1_1, 1) :
  - Feature: avg_jvm_heap_committed_value

(trace_id, ano_id) = (1_1, 2) :
  - Feature: avg_jvm_heap_committed_value

(trace_id, ano_id) = (1_1, 3) :
  - Feature: avg_jvm_heap_committed_value
  - Feature: avg_jvm_direct_used_value
  - Feature: avg_jvm_pools_PS-Survivor-Space_used_value

(trace_id, ano_id) = (1_1, 4) :
  - Feature: avg_jvm_pools_Metaspace_used_value

(trace_id, ano_id) = (1_1, 5) :
  - Feature: avg_jvm_pools_Metaspace_used_value


In [68]:
anomalies_features = organize_anomalies_features(final_features)

{('1_1', 0): ['avg_jvm_pools_PS-Old-Gen_max_value', 'avg_jvm_pools_PS-Old-Gen_usage_value', 'avg_jvm_pools_Metaspace_usage_value', 'avg_jvm_direct_used_value', 'avg_jvm_pools_PS-Survivor-Space_used_value'], ('1_1', 1): ['avg_jvm_heap_committed_value'], ('1_1', 2): ['avg_jvm_heap_committed_value'], ('1_1', 3): ['avg_jvm_heap_committed_value', 'avg_jvm_direct_used_value', 'avg_jvm_pools_PS-Survivor-Space_used_value'], ('1_1', 4): ['avg_jvm_pools_Metaspace_used_value'], ('1_1', 5): ['avg_jvm_pools_Metaspace_used_value']}


In [69]:
trace1_2 = pd.read_csv("1_2.csv")

In [70]:
trace_id = 1_2

In [104]:
def select_best_features(feature_clusters, final_filtered_rewards):
    """
    Sélectionne la meilleure caractéristique par cluster.
    """
    final_features = {}

    for (trace_id, ano_id), clusters in feature_clusters.items():
        retained_features = []
        feature_rewards = dict(final_filtered_rewards.get((trace_id, ano_id), []))

        for cluster in clusters:
            # Sélectionner **uniquement** la meilleure feature par cluster
            best_feature = max(cluster, key=lambda f: feature_rewards.get(f, 0), default=None)
            if best_feature:
                retained_features.append(best_feature)

        final_features[(trace_id, ano_id)] = retained_features

    return final_features


In [106]:
import pandas as pd
import numpy as np

def process_full_pipeline(trace, labels, trace_id, sample_fraction=0.8):
    """
    Process features for anomalies in a sampled trace, compute rewards, apply filtering,
    compute correlation, detect clusters, and select final features after clustering.
    
    This version performs sampling at 80% inside each anomaly's reference and anomalous period.
    
    Args:
        trace (pd.DataFrame): The full trace dataset.
        labels (pd.DataFrame): The anomaly labels for the dataset.
        trace_id (str): The ID of the trace being processed.
        sample_fraction (float): Fraction of each anomaly's segment to keep (default is 0.8).
    
    Returns:
        dict: Final retained features for each (trace_id, ano_id) after clustering.
    """
    print(f"\n🚀 STARTING PROCESS FOR TRACE: {trace_id}")

    # ✅ Étape 1: Filtrer les labels de la bonne trace
    filtered_labels = labels[labels["trace_id"] == trace_id].copy()
    print(f"\n📝 Filtered Labels for Trace {trace_id}:")
    print(filtered_labels)

    if filtered_labels.empty:
        print(f"❌ No anomalies found for trace {trace_id}. Skipping processing.")
        return {}

    # ✅ Étape 2: Échantillonnage à 80% des périodes de référence et anormales
    updated_labels = []
    for i, row in filtered_labels.iterrows():
        ano_id = row["ano_id"]
        ref_start, ref_end = int(row["ref_start"]), int(row["ref_end"])
        ano_start, ano_end = int(row["ano_start"]), int(row["ano_end"])

        # Sample 80% of the normal period
        normal_indices = np.linspace(ref_start, ref_end-1, num=(ref_end - ref_start), dtype=int)
        sampled_normal_indices = np.random.choice(normal_indices, size=int(len(normal_indices) * sample_fraction), replace=False)
        sampled_normal_indices.sort()
        
        # Update ref_end_post_sampling to last sampled index
        ref_end_post_sampling = sampled_normal_indices[-1]

        # Sample 80% of the anomalous period
        abnormal_indices = np.linspace(ano_start, ano_end-1, num=(ano_end - ano_start), dtype=int)
        sampled_abnormal_indices = np.random.choice(abnormal_indices, size=int(len(abnormal_indices) * sample_fraction), replace=False)
        sampled_abnormal_indices.sort()
        
        # Update ano_start and ano_end
        ano_start_post_sampling = ref_end_post_sampling  # Anomaly starts right after last normal index
        ano_end_post_sampling = sampled_abnormal_indices[-1]

        updated_labels.append({
            "trace_id": trace_id,
            "ano_id": ano_id,
            "ref_start": ref_start,  # ref_start remains unchanged
            "ref_end": ref_end_post_sampling,
            "ano_start": ano_start_post_sampling,
            "ano_end": ano_end_post_sampling,
            "ano_type": row["ano_type"]
        })

    updated_labels = pd.DataFrame(updated_labels)
    print(f"\n🔄 Updated Labels After Sampling:")
    print(updated_labels)

    # ✅ Étape 3: Calculer les récompenses pour chaque caractéristique dans chaque anomalie
    rewards = compute_rewards(trace, updated_labels, trace_id)
    print(f"\n🏆 Rewards Computed:")
    print(rewards)

    # ✅ Étape 4: Filtrer et trier les récompenses
    filtered_rewards = filter_and_sort_rewards(rewards)
    print(f"\n🔍 Filtered and Sorted Rewards:")
    print(filtered_rewards)

    # ✅ Étape 5: Appliquer le filtrage des récompenses avec la méthode du coude (Elbow Method)
    final_filtered_rewards, features_per_anomaly, _ = apply_reward_filtering(filtered_rewards)
    print(f"\n✅ Final Filtered Rewards:")
    print(final_filtered_rewards)

    # ✅ Étape 6: Extraire les caractéristiques par anomalie
    features_per_anomaly = extract_features_per_anomaly(final_filtered_rewards)
    print(f"\n📌 Features Per Anomaly Before Clustering:")
    print(features_per_anomaly)

    # ✅ Étape 7: Calculer les matrices de corrélation
    correlation_matrices = compute_correlation_matrices(features_per_anomaly, {})
    print(f"\n📊 Correlation Matrices:")
    print(correlation_matrices)

    # ✅ Étape 8: Construire les graphes de corrélation
    correlation_graphs = build_correlation_graphs(correlation_matrices)
    print(f"\n📊 Correlation Graphs:")
    print(correlation_graphs)

    # ✅ Étape 9: Détecter les clusters de caractéristiques
    feature_clusters = detect_feature_clusters(correlation_graphs)
    print(f"\n📌 Feature Clusters:")
    print(feature_clusters)

    # ✅ Étape 10: Sélectionner les meilleures caractéristiques par cluster
    final_features = select_best_features(feature_clusters, final_filtered_rewards)
    print(f"\n✅ Final Features Selected:")
    print(final_features)

    # ✅ Étape 11: Organiser les caractéristiques finales par anomalie
    anomalies_features = organize_anomalies_features(final_features)
    print(f"\n📊 Anomalies Features (Final Output):")
    print(anomalies_features)

    print(f"\n🎯 PROCESS COMPLETED FOR TRACE: {trace_id}\n")

    return anomalies_features

# ✅ Exécution avec l'échantillonnage mis à jour
final_features_1_2 = process_full_pipeline(trace1_2, labels, "1_2", sample_fraction=0.8)



🚀 STARTING PROCESS FOR TRACE: 1_2

📝 Filtered Labels for Trace 1_2:
    Unnamed: 0 trace_id  ano_id  ref_start  ref_end  ano_start  ano_end  \
6            6      1_2       0       5055     6989       6989     7956   
7            7      1_2       1      13025    14989      14989    15971   
8            8      1_2       2      21043    22989      22989    23962   
9            9      1_2       3      29033    30989      30989    31967   
10          10      1_2       4      37045    38989      38989    39961   

        ano_type  
6   bursty_input  
7   bursty_input  
8   bursty_input  
9   bursty_input  
10  bursty_input  

🔄 Updated Labels After Sampling:
  trace_id  ano_id  ref_start  ref_end  ano_start  ano_end      ano_type
0      1_2       0       5055     6988       6988     7955  bursty_input
1      1_2       1      13025    14987      14987    15969  bursty_input
2      1_2       2      21043    22988      22988    23961  bursty_input
3      1_2       3      29033    30988  

In [111]:
trace2_1 = pd.read_csv("2_1.csv")

In [113]:
final_features_2_1 = process_full_pipeline(trace2_1, labels, "2_1")


🚀 STARTING PROCESS FOR TRACE: 2_1

📝 Filtered Labels for Trace 2_1:
    Unnamed: 0 trace_id  ano_id  ref_start  ref_end  ano_start  ano_end  \
11          11      2_1       0       6954     8908       8908     9885   
12          12      2_1       1      16856    18808      18808    19784   
13          13      2_1       2      26756    28708      28708    29684   
14          14      2_1       3      36659    38609      38609    39584   

         ano_type  
11  stalled_input  
12  stalled_input  
13  stalled_input  
14  stalled_input  

🔄 Updated Labels After Sampling:
  trace_id  ano_id  ref_start  ref_end  ano_start  ano_end       ano_type
0      2_1       0       6954     8907       8907     9883  stalled_input
1      2_1       1      16856    18807      18807    19783  stalled_input
2      2_1       2      26756    28707      28707    29683  stalled_input
3      2_1       3      36659    38607      38607    39583  stalled_input

📌 Computing rewards for trace: 2_1
📝 Filtered labe

<!-- Feature Consistency Entropy Computation -->
<div style="background: linear-gradient(to right, #E67E22, #D35400); padding: 15px;">
    <h4 style="color: white;"><strong>Feature Consistency Entropy Computation</strong></h4>
    <p style="color: white;">
        In this step, we evaluate the stability of feature selection by computing the entropy of selected features across multiple subsampling runs.
        The goal is to measure how consistently a feature is chosen across different sampled datasets.
    </p>
    <ul style="color: white; font-size: 1em;">
        <li><strong>Subsampling the dataset:</strong> We randomly sample 80% of the dataset over multiple runs.</li>
        <li><strong>Executing the full feature selection pipeline:</strong> Each run executes the feature selection process to identify retained features.</li>
        <li><strong>Tracking feature occurrences:</strong> We store the selected features for each anomaly over multiple runs.</li>
        <li><strong>Computing feature entropy:</strong> The frequency of feature selection across runs is used to compute entropy, indicating selection consistency.</li>
        <li><strong>Output:</strong> The computed entropy values help in assessing feature stability, with lower entropy indicating higher consistency.</li>
    </ul>
</div>


In [92]:
import contextlib
import io
from collections import defaultdict
import numpy as np
from scipy.stats import entropy

def compute_feature_consistency_entropy_silent(trace, labels, trace_id, num_samples=5, sample_fraction=0.8):
    all_sampled_features = defaultdict(list)
    
    for i in range(num_samples):
        print(f"Executing run {i+1}/{num_samples}") 
        
        with contextlib.redirect_stdout(io.StringIO()):  # Suppress all other prints
            final_features = process_full_pipeline(trace, labels, trace_id, sample_fraction=sample_fraction)

        # Store the features selected for each anomaly in this run
        for (trace_id, ano_id), features in final_features.items():
            feature_set = set(features) 
            all_sampled_features[(trace_id, ano_id)].append(feature_set)

    # Compute entropy for feature consistency
    feature_entropies = {}
    for (trace_id, ano_id), feature_lists in all_sampled_features.items():
        feature_counts = defaultdict(int)
        total_runs = len(feature_lists)

        # Count occurrences of each feature across all runs
        for feature_set in feature_lists:
            for feature in feature_set:
                feature_counts[feature] += 1

        # Compute entropy
        probabilities = np.array(list(feature_counts.values())) / total_runs
        feature_entropy = entropy(probabilities, base=2)  # Compute entropy in base 2
        feature_entropies[(trace_id, ano_id)] = feature_entropy

    return feature_entropies

# ✅ Exécution avec correction
entropy_value = compute_feature_consistency_entropy_silent(trace1_1, labels, "1_1")
print(entropy_value)  # Only print the final entropy values

Executing run 1/5
Executing run 2/5
Executing run 3/5
Executing run 4/5
Executing run 5/5
{('1_1', 0): 1.2516291673878228, ('1_1', 1): 0.0, ('1_1', 2): 0.0, ('1_1', 3): 0.0, ('1_1', 4): 0.0, ('1_1', 5): 0.0}


<!-- Feature Consistency Entropy Computation -->
<div style="background: linear-gradient(to right, #E67E22, #D35400); padding: 15px;">
    <h4 style="color: white;"><strong>folder_1_results.csv generation</strong></h4>
    <p style="color: white;">
        For each anomaly, we store the trace ID, anomaly ID, explanation size, instability score, and selected feature indices.
        All computed metrics are aggregated into a CSV file.
    </p>
</div>


In [94]:
import pandas as pd
import numpy as np
import os
import contextlib
import io
from tqdm import tqdm  # Import tqdm for progress bar
from collections import defaultdict
from scipy.stats import entropy

# ✅ Charger les labels
labels = pd.read_csv('labels.csv')

# ✅ Extraire les IDs uniques des traces
trace_ids = labels["trace_id"].unique()

# ✅ Fonction pour générer les explications et mesurer l'instabilité
def generate_explanation_metrics(trace_id, labels, num_samples=5, sample_fraction=0.8):
    """
    Compute explanations and instability metrics for a given trace.

    Args:
        trace_id (str): The trace identifier.
        labels (pd.DataFrame): The anomaly labels DataFrame.
        num_samples (int): Number of sampling runs for entropy computation.
        sample_fraction (float): Fraction of the dataset sampled per run.

    Returns:
        list: List of dictionaries containing explanation metrics.
    """
    trace_path = f"{trace_id}.csv"
    if not os.path.exists(trace_path):
        print(f"❌ Warning: Trace file {trace_path} not found. Skipping.")
        return []

    trace = pd.read_csv(trace_path)  # Charger les données de la trace

    # ✅ Calcul de l'entropie d'instabilité pour chaque anomalie
    entropy_values = compute_feature_consistency_entropy_silent(
        trace, labels, trace_id, num_samples=num_samples, sample_fraction=sample_fraction
    )

    results = []
    for (trace_id, ano_id), entropy_value in tqdm(entropy_values.items(), desc=f"Processing {trace_id}", leave=False):
        
        # ✅ Exécuter process_full_pipeline pour obtenir les caractéristiques finales
        with contextlib.redirect_stdout(io.StringIO()):  
            final_features = process_full_pipeline(trace, labels, trace_id, sample_fraction=sample_fraction)
        
        # Récupérer les caractéristiques finales utilisées pour l'explication
        selected_features = final_features.get((trace_id, ano_id), [])

        # ✅ Convertir les noms de caractéristiques en indices
        feature_indices = [trace.columns.get_loc(feature) for feature in selected_features if feature in trace.columns]

        # ✅ Stocker les résultats
        results.append({
            "trace_id": trace_id,
            "ano_id": ano_id,
            "exp_size": len(selected_features),
            "exp_instability": entropy_value,
            "explanation": ', '.join(map(str, feature_indices))  # Stocker les indices des features sous forme de chaîne
        })

    return results

# ✅ Générer les explications et les métriques pour toutes les traces
all_results = []
for trace_id in tqdm(trace_ids, desc="Processing all traces"):
    all_results.extend(generate_explanation_metrics(trace_id, labels))

# ✅ Convertir en DataFrame et sauvegarder les résultats
results_df = pd.DataFrame(all_results)
results_df.to_csv("folder_1_results.csv", index=False)

print("\n✅ Processing complete! Results saved in folder_1_results.csv")

Processing all traces:   0%|                              | 0/6 [00:00<?, ?it/s]

Executing run 1/5
Executing run 2/5
Executing run 3/5
Executing run 4/5
Executing run 5/5



Processing all traces:  17%|███▋                  | 1/6 [00:53<04:28, 53.80s/it]

Executing run 1/5
Executing run 2/5
Executing run 3/5
Executing run 4/5
Executing run 5/5



Processing all traces:  33%|███████▎              | 2/6 [01:35<03:05, 46.46s/it]

...

Executing run 1/5
Executing run 2/5
Executing run 3/5
Executing run 4/5
Executing run 5/5



Processing all traces:  83%|██████████████████▎   | 5/6 [03:12<00:36, 36.67s/it]

Executing run 1/5
Executing run 2/5
Executing run 3/5
Executing run 4/5
Executing run 5/5



Processing all traces: 100%|██████████████████████| 6/6 [03:55<00:00, 39.28s/it]


✅ Processing complete! Results saved in folder_1_results.csv
